<a name="top"></a>
<p style="text-align: right"><a href="#bottom" style="text-decoration: none;color: pink"><strong>Bottom</strong></a></p>

# <center>DP

In [1]:
public class TreeNode {
    int val;
    TreeNode left;
    TreeNode right;
    TreeNode(int x) { val = x; }
}

## [198. House Robber (easy)](https://leetcode-cn.com/problems/house-robber/)

You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed, the only constraint stopping you from robbing each of them is that adjacent houses have security system connected and **it will automatically contact the police if two adjacent houses were broken into on the same night.**

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight **without alerting the police.**

<font color='dd0000'>

状态设计：$dp[i]$ 表示 在前 $i$ 家能抢到的最大金额

状态转移：$dp[i] = max(dp[i-1], dp[i-2] + nums[i])$

</font>

In [2]:
public int rob(int[] nums) {
    int n = nums.length;
    if(n < 1){
        return 0;
    }
    int[] dp = new int[n + 1];
    dp[1] = nums[0];  // 初始化
    for(int i = 2; i <= n; i++){
        dp[i] = Math.max(dp[i-1], dp[i-2] + nums[i - 1]);  // 注意i-1
    }
    return dp[n];
}

## [213. House Robber II (medium)](https://leetcode-cn.com/problems/house-robber-ii/)

You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed. All houses at this place are **arranged in a circle**. That means the first house is the neighbor of the last one. Meanwhile, adjacent houses have security system connected and **it will automatically contact the police if two adjacent houses were broken into on the same night.**

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight **without alerting the police.**

<font color='dd0000'>

状态设计和状态转移同上，区别为第一位和最后一位不能同时选，所以分成【选第一个】和【不选第一个】两种情况，最后取两者较大的值。

</font>

In [3]:
public int rob(int[] nums) {
    int n = nums.length;
    if(n < 1){
        return 0;
    }
    // 选第一个
    int[] d = new int[n + 1];
    d[1] = nums[0];
    for(int i = 2; i <= n; i++){
        if(i != n){
            d[i] = Math.max(d[i-1], d[i-2] + nums[i-1]);
        } else {
            d[i] = d[i-1];  // 此时最后一个不能选
        }
    }
    // 不选第一个
    int[] p = new int[n + 1];
    for(int i = 2; i <= n; i++){
        p[i] = Math.max(p[i-1], p[i-2] + nums[i-1]);  //此时最后一个可选可不选
    }
    return Math.max(d[n], p[n]);
}

## ⭐[337. House Robber III (medium) [Tree and DP]](https://leetcode-cn.com/problems/house-robber-iii/)

The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". **It will automatically contact the police if two directly-linked houses were broken into on the same night.**

Determine the maximum amount of money the thief can rob tonight without alerting the police.

<font color='dd0000'>取得最大金额的情况为：
    
1. 不选根节点：***opt(root) = opt(root.left) + opt(root.right)***


2. 选根节点：  ***opt(root) = root + opt(left.left) + opt(left.right) + opt(right.left) + opt(right.right)***

根据上述递推式可构造出递归
</font>

In [2]:
// 解法1：递归 重复计算重叠子问题 复杂度高

public int rob(TreeNode root) {
    if(root == null) return 0;
    int a = root.val;
    if(root.left != null) a += rob(root.left.left) + rob(root.left.right);
    if(root.right != null) a += rob(root.right.left) + rob(root.right.right);
    int b = rob(root.left) + rob(root.right);
    return Math.max(a, b);
}

In [4]:
// 解法2：记忆化递归

// 通过递归，在递归过程中把结果逐渐放入哈希表（备忘录）
// 遇到已计算过的结点直接返回
public int rob22(TreeNode root){
    Map<TreeNode, Integer> opt = new HashMap<>();
    return helper(root, opt);
}

private int helper(TreeNode root, Map<TreeNode, Integer> opt){
    if(root == null) return 0;
    if(opt.containsKey(root)) return opt.get(root);  // 先序遍历，根结点是最后处理的，当表里已经有root了，说明已经遍历完成，返回它的结果
    
    int a = root.val;
    if(root.left != null) a += helper(root.left.left, opt) + helper(root.left.right, opt);
    if(root.right != null) a += helper(root.right.left, opt) + helper(root.right.right,opt);  // 选
    
    int b = helper(root.left, opt) + helper(root.right, opt);  // 不选
    
    int max = Math.max(a, b);
    opt.put(root, max);      // 在表中记录当前结果
    return max;      // 返回值是每个结点能取到的最大值
}

<font color='dd0000'>从另一个角度考虑，每个结点有选/不选两个状态，每个结点能取的最大值的情况为：

1. 选：两个子节点不能选，那么最大值为 "**不选左子结点时，**左子树能取到的最大值 + **不选右子节点时，**右子节点能取到的最大值 + 自己本身的值"


2. 不选：最大值为 "**左子树在选/不选中的最大值** + **右子树在选/不选中的最大值**"

所以，可以用一个大小为2的数组，分别保存每个节点选或不选两个状态的最大值

这样就不必考虑到孙子节点
</font>

In [8]:
// 解法3：res[0]保存不选该节点时的最大值，res[1]表示选该节点时的最大值

public int rob3(TreeNode root){
    int a = solve(root)[0];
    int b = solve(root)[1];
    return Math.max(a, b);
}

public int[] solve(TreeNode root){
    if(root == null) return new int[2];
    
    int[] res = new int[2];
    
    res[0] = Math.max(solve(root.left)[0], solve(root.left)[1]) + Math.max(solve(root.right)[0], solve(root.right)[1]);
    res[1] = root.val + solve(root.left)[0] + solve(root.right)[0];
    
    return res;
}

## [32. Longest Valid Parentheses (hard)](https://leetcode-cn.com/problems/longest-valid-parentheses/)

Given a string containing just the characters ```'('``` and ```')'```, find the length of the longest valid (well-formed) parentheses substring.

In [1]:
// 状态设计：dp[i]表示以第i个字符结尾的最长有效长度
// 有：dp[0]一定为0; 
//     若arr[i]=='(',则dp[i]=0
// 当arr[i]==')'时，如果【i-dp[i-1]-1】位置正好是'('，那么dp[i] = dp[i-1]+2+dp[i-dp[i - 1]-1 - 1];否则，dp[i] = 0
// 最后，最大长度就是dp中最大的数值

public int longestValidParentheses(String s) {
    if(s == null || s.length() == 0)
            return 0;
        
    char[] arr = s.toCharArray();
    int[] dp = new int[s.length()];
    // dp[i]表示以字符arr[i]结尾的最大有效长度
    int res = 0;  // 保存最大值
    for(int i = 1; i < arr.length; i++){
        if(arr[i] == ')'){
            int pre = i - dp[i - 1] - 1;
            if(pre >= 0 && arr[pre] == '(')
                dp[i] = dp[i - 1] + 2 + ((pre - 1 >= 0) ? dp[pre - 1] : 0);
        }
        res = Math.max(res, dp[i]);
    }
    return res;
}

## [70. Climbing Stairs (easy)](https://leetcode-cn.com/problems/climbing-stairs/)

You are climbing a stair case. It takes *n* steps to reach to the top.

Each time you can either climb 1 or 2 steps. In how many distinct ways can you climb to the top?

**Note:** Given n will be a positive integer.

<font color='dd0000'>

状态设计：$dp[i]$ 表示走到第 $i$ 楼的方式数

状态转移：在每一层，可能是走1步上来的，可能是走2步上来的，所以有 $dp[i] = dp[i-1] + dp[i-2]$

</font>

In [6]:
public int climbStairs(int n) {
    int[] dp = new int[n + 1];
    dp[0] = 1;
    dp[1] = 1;
    for(int i = 2; i<= n; i++){
        dp[i] = dp[i-1] + dp[i-2];
    }
    return dp[n];
}

## [322. Coin Change (medium)](https://leetcode-cn.com/problems/coin-change/)

You are given coins of different denominations(面值) and a total amount of money *amount*. Write a function to compute the **fewest** number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return ```-1```.

<font color='dd0000'>**推导：完全背包**</font>

<font color='dd0000'>

问题可以转换为：每步可以上楼梯的阶数为coins数组里的值，用最少的步数上到amount楼。

**状态设计**：```dp[i]``` 表示走到第 *i* 楼的最小步数，初始状态为```dp[0] = 0```

**状态转移方程**：在第```i```步，能上的层数有```coins[1]、coins[2]、……```，所以```dp[i] = dp[i - coins[j]] + 1(当前这一步)```，从```dp[i - coins[j]]```中选择最小的那个，就是```dp[i]```的最小值，即：```dp[i] = min{dp[i - coins[j]} + 1```。

**注意点**：因为更新```dp[i]```时是不断取最小的，所以初值应设一个大的值，因为最大就是走到```amount```，所以设为```amount+1```很合适(```dp[0]```除外)。并且，会存在某一步没有解的情况，那么```dp[i]```的值就会始终是```amount+1```，最后可通过这个值来判断最后一步有没有解。
</font>

In [1]:
public int coinChange(int[] coins, int amount) {
    int[] dp = new int[amount + 1]; // 从0~amount,所以amount+1位的数组
    for(int k = 1; k < dp.length; k++)
        dp[k] = amount + 1;         // 设初值
    for(int i = 1; i < dp.length; i++){
        for(int j = 0; j < coins.length; j++){ // 检查每一步能上的阶数
            if(i - coins[j] >= 0){
                dp[i] = Math.min(dp[i], dp[i - coins[j]] + 1); // 不断更新最小值,如果没有更好的解,就保持初值amount+1
            }
        }
    }
    return dp[amount] > amount ? -1 : dp[amount];  // 如果dp[i]>amount,说明无解,返回-1；否则返回代表最优解的dp[i]
}

<font color='dd0000'>解法二：**贪心策略 + dfs搜索 + 剪枝**</font>

<font color='dd0000'>应用贪心策略的暴力dfs搜索策略为：从最大面值开始，先选最多的最大面值，总额减去当前已选择面额，再选最多的次大面值，依次类推，直到最小的面值求出一个解或无解，然后回溯到每一个大的面值处，将选择数量-1，不断更新解。但是这样将遍历所有的可能组合，复杂度很高。

但其实观察搜索树可知：**当搜索到某一层，发现此时的解(已选的个数)已经大于当前最优解了，那么就可以不再往下继续搜索了，实现剪枝**
</font>

In [1]:
public int coinChange2(int[] coins, int amount) {
    Arrays.sort(coins); // 必须对数组排好序，才能按从大到小的面值依次搜索
    dfs(coins, coins.length - 1, amount, 0);
    return ans == Integer.MAX_VALUE ? -1 : ans;
}

private int ans = Integer.MAX_VALUE; // 因为要更新最小值，所以解的初值设为一个大数
private void dfs(int[] coins, int index, int amount, int count){ // index表示当前选择的面值的索引；count表示当前已经选了的个数
    int coin = coins[index];
    if(index == 0){   // base case:搜索到最小面值
        if(amount % coin == 0){  // 如果剩余额度正好可以整除当前面值
            ans = Math.min(ans, count + amount / coin);  // 更新解
        }
    }
    else{                                                            //k表示每次选择当前面值的个数,从最多开始逐渐减到0;
        for(int k = amount / coin; k >= 0 && count + k < ans; k--){ //【count+k<ans】控制剪枝:
            dfs(coins, index - 1, amount - coin * k, count + k);   //只有【已选个数count+当前选的个数k】<【当前最优解】时才继续搜索
        }                                                         //继续搜索时,代表面值的索引-1,总额度-当前已选面额,总个数+当前已选个数
    }
}

## [518. Coin Change 2 (medium)](https://leetcode-cn.com/problems/coin-change-2/)

You are given coins of different denominations and a total amount of money. Write a function to compute the number of combinations that make up that amount. You may assume that you have infinite number of each kind of coin.

<font color='dd0000'>
    
本质上是【**完全背包问题**】

状态设计：二维dp，$dp[i][j]$ 表示使用前 $i$ 种硬币凑出 $j$ 元的方法数

状态转移：$dp[i][j] = dp[i-1][j] + dp[i][j - coins[i]]$

优化：二维数组优化为一维数组

（详细推导见笔记）
</font>

In [3]:
// 二维dp
public int change(int amount, int[] coins) {
    if(amount == 0){
        return 1;
    }
    if(coins.length < 1){
        return 0;
    }
    
    int[][] dp = new int[coins.length][amount + 1];
    // 初始化
    for(int j = 0; j < dp[0].length; j++){
        if(j % coins[0] == 0){
            dp[0][j] = 1;
        }
    }
    for(int i = 0; i < dp.length; i++){
        dp[i][0] = 1;
    }
    // 递推
    for(int i = 1; i < dp.length; i++){
        for(int j = 1; j < dp[i].length; j++){
            dp[i][j] = dp[i - 1][j]; // 不选第i个硬币已有的组合数
            if(j - coins[i] >= 0){
                dp[i][j] += dp[i][j - coins[i]]; // 再加上选第i个硬币的组合数
            }
        }
    }
    return dp[coins.length - 1][amount];
}

// 空间优化
public int change2(int amount, int[] coins) {
    int[] dp = new int[amount + 1];
    // 初始化
    dp[0] = 1;
    // 递推
    for(int coin : coins){
        for(int j = coin; j <= amount; j++){
            dp[j] += dp[j - coin];
        }
    }
    return dp[amount];
}

## [面试题 08.11. 硬币 (medium)](https://leetcode-cn.com/problems/coin-lcci/)

给定数量不限的硬币，币值为25分、10分、5分和1分，编写代码计算n分有几种表示法。(结果可能会很大，你需要将结果模上1000000007)

<font color='dd0000'>是上题的一种特殊情况</font>

In [4]:
// 优化的一维解法
public int waysToChange(int n) {
    int mod = 1000000007;
    int[] dp = new int[n + 1];
    // 初始化
    dp[0] = 1;
    int[] coins = {1, 5, 10, 25};
    for(int coin : coins){
        for(int j = coin; j <= n; j++){
            dp[j] += dp[j - coin];
            dp[j] %= mod;
        }
    }
    return dp[n];
}

## [279. Perfect Squares (medium)](https://leetcode-cn.com/problems/perfect-squares/)

Given a positive integer n, find the least number of perfect square numbers (for example, ```1, 4, 9, 16, ...```) which sum to n.

<font color='dd0000'>动态规划解法：将小于给定数的完全平方数都求出来放进一个数组，转换为类似上题的问题：用最小数量的数组里的数组成给定数</font>

In [4]:
public int numSquares(int n) {
    List<Integer> squares = new ArrayList<>();  // 求完全平方数组
    int x = 1;
    while(x * x <= n){
        squares.add(x * x);
        x++;
    }
        
    int[] dp = new int[n + 1];          // dp[]初始化
    for(int k = 1; k < dp.length; k++){
        dp[k] = n + 1;
    }
    for(int i = 1; i < dp.length; i++){
        for(int j = 0; j < squares.size(); j++){
            if(i - squares.get(j) >= 0){
                dp[i] = Math.min(dp[i], dp[i - squares.get(j)] + 1);
            }
        }
    }
    return dp[n] > n ? -1 : dp[n];
}

// 代码优化：思路相同，但是没有必要用一个数组来存这些完全平方数，可以直接通过内部的循环解决
public int numSquares2(int n) {
    int[] dp = new int[n + 1];
    for(int i = 1; i < dp.length; i++){
        dp[i] = i;  // 因为对每个i,最坏情况就是由i个1组成,所以为dp[i]赋初值为i
        for(int j = 1; j * j < i; j++){  // 用循环控制每一步都是小于i的完全平方数
            dp[i] = Math.min(dp[i], dp[i - j * j] + 1);
        }
    }
    return dp[n];  // dp[n]一定有解,无需判断
}

<font color='dd0000'>解法二：BFS。See Search:BFS</font>

## [121. Best Time to Buy and Sell Stock (easy) ](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock/)<font color='dd0000'>[1次交易]</font>

Say you have an array for which the *$i^{th}$* element is the price of a given stock on day i.

If you were only permitted to complete ***at most one transaction (i.e., buy one and sell one share of the stock)***, design an algorithm to find the maximum profit.

Note that you cannot sell a stock before you buy one.

<font color='dd0000'>**动态规划思想**：*dp[i]* 表示前 *i* 天的最大利润，那么在第 *i* 天, *dp[i]* 应等于【前*i-1*天的最大利润】和【第*i*天卖出的价格 - 前*i-1*天中的最小价格】中的最大值</font>

In [2]:
// 遍历数组：记录下最小的价格，每天看当天价格 - 最小价格能否成为最大利润

public int maxProfit(int[] prices) {
    if(prices.length <= 1) return 0;
    
    int min = prices[0]; // 保存最小价格
    int res = 0;
    for(int i = 0; i < prices.length; i++){
       if(prices[i] < min)
           min = prices[i];           // !!注意：if-else if的写法
       else if(prices[i] - min > res) // 因为若满足第一个条件必不可能再满足第二个条件,所以这样可以在每次循环中少进行一次判断
           res = prices[i] - min;
    }
    return res;
}

In [ ]:
// 滑动窗口思想

// 用双指针模拟双端队列
public int maxProfit2(int[] prices) {
    int res = 0;
    int i = 0, j = 1;         // i维护最小价格,j检查每天的价格
    while(j < prices.length){
        if(i == j){
            j++;
        }
        else if(prices[i] >= prices[j]){
            i++;
        }
        else if(prices[i] < prices[j]){
            res = Math.max(res, prices[j] - prices[i]);
            j++;
        }
    }
    return res;
}

// 直接用双端队列,维护一个单调队列
public int maxProfit3(int[] prices) {
    Deque<Integer> dq = new LinkedList<>();
    int res = 0;
    for(int i = 0; i < prices.length; i++){
        int price = prices[i];
        while(!dq.isEmpty() && price < dq.getLast()){
            dq.removeLast();
        }
        if(dq.isEmpty()){
            dq.addLast(price);
            continue;
        }
        res = Math.max(res, price - dq.getFirst());
        dq.addLast(price);
    }
    return res;
}

## [122. Best Time to Buy and Sell Stock II (easy)](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-ii/)<font color='dd0000'>[无限次交易]</font>

Say you have an array ```prices``` for which the $i^{th}$ element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete ***as many transactions as you like (i.e., buy one and sell one share of the stock multiple times)***.

**Note:** You may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

<font color='dd0000'>
    
**方法一：暴力DFS（超时）**

在每一天有两种状态：【有股票】和【没股票】，搜索全部可能的状态组合，搜索到最后一天时取结果

对于每一天：

1. 处于【有股票】状态时，可以进行的操作为【卖出】，此时下一天的状态就变为【没股票】；或者不进行操作，下一天的状态为【有股票】
        
2. 处于【没股票】状态时，可以进行的操作为【买进】，此时下一天的状态变为【有股票】；或者不进行操作，下一天的状态为【没股票】

</font>

In [2]:
// 暴力DFS
int res = 0;
public int maxProfit1(int[] prices) {
    dfs(prices, 0, false, prices.length, 0); // 初始状态：没股票，没利润
    return res;
}

/**
  prices: 股票价格数组
  i     : 表示当前操作第i天
  flag : 标记第i天的状态，true-有股票 false-没股票
  n     : 数组长度
  profit: 当前获得的总利润
*/
private void dfs(int[] prices, int i, boolean flag, int n, int profit){
    if(i == n){  // 出口：已经搜索完整个数组
        res = Math.max(res, profit);
        return;
    }
    if(flag){ // 有股票
        dfs(prices, i + 1, false, n, profit + prices[i]);  // 卖出：状态变为没股票，利润增加
        dfs(prices, i + 1, true, n, profit);  // 不操作：状态不变，利润不变
    } else { // 没股票
        dfs(prices, i + 1, true, n, profit - prices[i]); // 买进：状态变为有股票，利润减少
        dfs(prices, i + 1, false, n, profit);  // 不操作：状态不变，利润不变
    }
}

<font color='dd0000'>

**方法二：动态规划**

因为每一天有两个状态，每个状态有不同的操作，所以使用需要定义两个状态数组

状态定义：$d[i]$ 表示第 $i$ 天有股票时的利润；$p[i]$ 表示第 $i$ 天没股票时的利润

状态转移：$d[i] = max(d[i-1], p[i-1] - prices[i])$ ；$p[i] = max(p[i-1], d[i-1] + prices[i])$

最后可用滚动变量进行空间优化

</font>

In [3]:
 public int maxProfit21(int[] prices) {
    int n = prices.length;
    if(n < 1){
        return 0;
    }
    int[] d = new int[n]; //有股票
    int[] p = new int[n]; //没股票
    d[0] = -prices[0];
    p[0] = 0;
    for(int i = 1; i < n; i++){
        d[i] = Math.max(d[i-1], p[i-1] - prices[i]);
        p[i] = Math.max(p[i-1], d[i-1] + prices[i]);
    }
    return p[n-1]; // 最后的最大利润肯定是手里没股票的情况
}

// 空间优化
public int maxProfit22(int[] prices) {
    int n = prices.length;
    if(n < 1){
        return 0;
    }
    int d = -prices[0]; //有股票
    int p = 0; //没股票
    for(int i = 1; i < n; i++){
        int t = d;
        d = Math.max(d, p - prices[i]);
        p = Math.max(p, t + prices[i]);
    }
    return p;
}

<font color='dd0000'>

**方法三：贪心**

因为可以买卖无数次，所以只要第二天价格比当天价格高，就在当天买进第二天卖出

</font>

In [4]:
public int maxProfit3(int[] prices) {
    int res = 0;
    for(int i = 1; i < prices.length; i++){
        if(prices[i] > prices[i-1]){
            res += prices[i] - prices[i-1];
        }
    }
    return res;
}

## ⭐[188. Best Time to Buy and Sell Stock IV (hard)](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iv/)<font color='dd0000'>[k次交易]</font>

Say you have an array for which the i-th element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete at most **k** transactions.。

<font color='dd0000'>
    
将**买入一支股票**视为进行了一次交易，一个状态由**在第几天、进行了几次交易、不持有/持有股票**决定，因此这是三维dp

**状态定义：** 

$dp[i][k][0] / [1]$ 表示 *在前 $i$ 天最多进行了 $k$ 笔交易且不持有 / 持有股票的最大收益*

**状态转移：** 

$dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])$：不动，或者卖出

$dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])$：不动，或者买进

</font>

In [ ]:
// 朴素三维dp
public int maxProfit(int k, int[] prices) {
    int n = prices.length;
    
    if(k > n / 2){  // 因为对一支股票的一次交易至少要用两天，如果 k > n / 2，等同于允许交易无限次，转为122题，使用贪心计算
        int res = 0;
        for(int i = 1; i < n; i++){
            if(prices[i] > prices[i - 1]){
                res += prices[i] - prices[i - 1];
            }
        }
        return res;
    }
    
    int[][][] dp = new int[n + 1][k + 1][2]; // 对天数多设一个没有股票的第0天，方便处理边界
    // 初始化
    for(int i = 0; i <= n; i++){
        dp[i][0][1] = Integer.MIN_VALUE; // 不进行交易不可能持有股票，不可能用大负数表示，这样转移时不可能取到
    }
    for(int t = 0; t <= k; t++){
        dp[0][t][1] = Integer.MIN_VALUE; // 在第0天，无论进行多少次交易都不可能持有股票
    }
    // 递推
    for(int i = 1; i <= n; i++){
        for(int t = 1; t <= k; t++){
            dp[i][t][0] = Math.max(dp[i-1][t][0], dp[i-1][t][1] + prices[i-1]);
            dp[i][t][1] = Math.max(dp[i-1][t][1], dp[i-1][t-1][0] - prices[i-1]);
        }
    }
    return dp[n][k][0];
}

// 空间优化：因为每次的状态只和前一天有关，所以对 i 这一维可去掉；对是否持有股票这一维，可使用两个数组，所以将状态压缩为一个数组
public int maxProfit(int k, int[] prices) {
    int n = prices.length;
    if(k > n / 2){
        int res = 0;
        for(int i = 1; i < n; i++){
            if(prices[i] > prices[i-1]){
                res += prices[i] - prices[i-1];
            }
        }
        return res;
    }
    
    int[] d = new int[k + 1]; // 没股票
    int[] p = new int[k + 1]; // 有股票
    // 初始化
    for(int t = 0; t <= k; t++){
        p[t] = Integer.MIN_VALUE;  // 第0天不可能持有股票
    }
    // 递推
    for(int price : prices){
        for(int t = k; t >= 1; t--){ // 要用到上一层t-1的值，因此倒着遍历，避免覆盖
            d[t] = Math.max(d[t], p[t] + price);
            p[t] = Math.max(p[t], d[t - 1] - price);
        }
    }
    return d[k];
 }

## [123. Best Time to Buy and Sell Stock III (hard)](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iii/) <font color='dd0000'>[2次交易]</font>

Say you have an array for which the $i^{th}$ element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete **at most two transactions.**

<font color='dd0000'>k次交易的特殊情况，令 k=2 即可</font>

In [1]:
public int maxProfit(int[] prices) {
    int n = prices.length;
    int k = 2;
    int[] d = new int[k + 1];
    int[] p = new int[k + 1];
    // 初始化
    for(int i = 0; i <= k; i++){
        p[i] = Integer.MIN_VALUE;
    }
    for(int i = 0; i < n; i++){
        for(int t = k; t > 0; t--){
            d[t] = Math.max(d[t], p[t] + prices[i]);
            p[t] = Math.max(p[t], d[t - 1] - prices[i]);
        }
    }
    return d[k];
}

## [309. Best Time to Buy and Sell Stock with Cooldown (medium) ](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/)<font color='dd0000'>[含冷冻期]</font>

Say you have an array for which the $i^{th}$ element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete as many transactions as you like (ie, buy one and sell one share of the stock multiple times) with the following restrictions:

- You may not engage in multiple transactions at the same time (ie, you must sell the stock before you buy again).
- **After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)**

<font color='dd0000'>等同于无限次交易，含冷冻期，稍微修改状态转移方程即可</font>

In [2]:
public int maxProfit(int[] prices) {
    int n = prices.length;
    if(n <= 1){
        return 0;
    }
    int[] d = new int[n]; // 没股票
    int[] p = new int[n]; // 有股票
    p[0] = -prices[0];
    for(int i = 1; i < n; i++){
        d[i] = Math.max(d[i - 1], p[i - 1] + prices[i]);
        if(i == 1){
            p[i] = Math.max(p[i - 1], d[0] - prices[i]); // 特殊考虑
        } else {
            p[i] = Math.max(p[i - 1], d[i - 2] - prices[i]); // 虽然d[i-1]不一定是卖出了股票导致需要冷却
        }                                                    // 但这种情况(第i-1天没有卖出)相当于是从d[i-2]转移过来，利润都是同一个值
    }                                                        // 所以取d[i-2]是正确的
    return d[n - 1];
}

## [714. Best Time to Buy and Sell Stock with Transaction Fee (medium)](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-with-transaction-fee/) <font color='dd0000'>[含手续费]</font>

Your are given an array of integers ```prices```, for which the i-th element is the price of a given stock on day i; and a non-negative integer ```fee``` representing a transaction fee.

You may complete as many transactions as you like, but you need to pay the transaction fee for each transaction. You may not buy more than 1 share of a stock at a time (ie. you must sell the stock share before you buy again.)

Return the maximum profit you can make.

<font color='dd0000'>等同于无限次交易，交易时需要额外支出手续费，稍微修改状态转移方程即可</font>

In [3]:
// 已进行空间优化
public int maxProfit(int[] prices, int fee) {
    int n = prices.length;
    if(n <= 1){
        return 0;
    }
    int d = 0; // 没股票
    int p = -prices[0]; // 有股票
    for(int i = 1; i < n; i++){
        int t = d; // 先保存d，避免修改
        d = Math.max(d, p + prices[i] - fee); // 卖出股票时减去手续费
        p = Math.max(p, t - prices[i]);
    }
    return d;
}

## [300. Longest Increasing Subsequence (medium)](https://leetcode-cn.com/problems/longest-increasing-subsequence/)

Given an unsorted array of integers, find the length of longest increasing subsequence.

<font color='dd0000'>动态规划解法，复杂度为 $O(n^2)$。最优解法为二分查找，见 ***Binary Search***</font>

In [1]:
// DP：状态设计：dp[i] 表示以第i个数结尾的最大上升子序列的长度
//     初始值：  dp[]数组初始化为每个值都是1
//     状态转移：对于第i个数，遍历它之前的所有数，如果它比第j个数大，那么dp[i]=dp[j] + 1，从中取最大的就是dp[i]的值

public int lengthOfLIS(int[] nums) {
    if(nums.length < 1)
        return 0;
    
    int[] dp = new int[nums.length];
    for(int i = 0; i < dp.length; i++){ // 初始化
        dp[i] = 1;
    }
    int ans = 0;
    for(int i = 0; i < nums.length; i++){
        for(int j = 0; j < i; j++){
            if(nums[i] > nums[j]){
                dp[i] = Math.max(dp[i], dp[j] + 1); 
            }
        }
        ans = Math.max(ans, dp[i]);
    }
    return ans;
}

## ⭐[72. Edit Distance (hard)](https://leetcode-cn.com/problems/edit-distance/)

Given two words word1 and word2, find the minimum number of operations required to convert *word1* to *word2*.

You have the following 3 operations permitted on a word:

1. Insert a character
2. Delete a character
3. Replace a character

<font color='dd0000'>

状态设计：$dp[i][j]$: 表示从```word1```的前$i$个字符编辑成```word2```的前$j$个字符需要的操作数

状态转移：

1. 如果```word1```的第$i$个字符和```word2```的第$j$个字符相同，那么当前这步不需要操作，$dp[i][j] = dp[i-1][j-1]$


2. 否则，每一步有三种编辑方式：**insert**: $dp[i][j] = dp[i][j-1] + 1$、**delete**: $dp[i][j] = dp[i-1][j] + 1$、**replace**: $dp[i][j] = dp[i-1][j-1] + 1$

所以取上述三个值中最小的那个
</font>

In [2]:
 public int minDistance(String word1, String word2) {
    int m = word1.length();
    int n = word2.length();
    int[][] dp = new int[m + 1][n + 1]; // 长度从0到length，所以数组大小+1
    for(int i = 0; i <= m; i++) dp[i][0] = i;
    for(int j = 0; j <= n; j++) dp[0][j] = j;
    for(int i = 1; i <= m; i++){
        for(int j = 1; j <= n; j++){
            if(word1.charAt(i - 1) == word2.charAt(j - 1)){ // 对应字符下标
                dp[i][j] = dp[i-1][j-1];
            } else {
                dp[i][j] = Math.min(dp[i-1][j], Math.min(dp[i][j-1], dp[i-1][j-1])) + 1;
            }
        }
    }
    return dp[m][n];
}

## [62. Unique Paths (medium)](https://leetcode-cn.com/problems/unique-paths/)

A robot is located at the top-left corner of a ```m x n``` grid.

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid.

How many possible unique paths are there?

<font color='dd0000'>
    
状态设计：$dp[i][j]$ 表示在 $(i, j)$ 位置的路径数

状态转移：$dp[i][j] = dp[i-1][j] + dp[i][j-1]$

</font>

In [4]:
public int uniquePaths(int m, int n) {
    int[][] dp = new int[m][n];
    //初始化
    for(int j = 0; j < n; j++){
        dp[0][j] = 1;
    }
    for(int i = 0; i < m; i++){
        dp[i][0] = 1;
    }
    for(int i = 1; i < m; i++){
        for(int j = 1; j < n; j++){
            dp[i][j] = dp[i-1][j] + dp[i][j-1];
        }
    }
    return dp[m-1][n-1];
}

## [63. Unique Paths II (medium)](https://leetcode-cn.com/problems/unique-paths-ii/)

A robot is located at the top-left corner of a ```m x n``` grid.

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid.

Now consider if some obstacles are added to the grids. How many unique paths would there be?

<font color='dd0000'>状态设计和状态转移同上题，加上遇到障碍的处理</font>

In [5]:
public int uniquePathsWithObstacles(int[][] obstacleGrid) {
    int m = obstacleGrid.length;
    int n = obstacleGrid[0].length;
    int[][] dp = new int[m][n];
    // 初始化
    for(int j = 0; j < n; j++){
        if(obstacleGrid[0][j] == 1){ // 遇到障碍后面的位置均不可达
            break;
        }
        dp[0][j] = 1;
    }
    for(int i = 0; i < m; i++){
        if(obstacleGrid[i][0] == 1){
            break;
        }
        dp[i][0] = 1;
    }
    for(int i = 1; i < m; i++){
        for(int j = 1; j < n; j++){
            if(obstacleGrid[i][j] != 1) {
                dp[i][j] = dp[i-1][j] + dp[i][j-1];
            }
        }
    }
    return dp[m-1][n-1];
}

## [120. Triangle (medium)](https://leetcode-cn.com/problems/triangle/)

Given a triangle, find the minimum path sum from top to bottom. Each step you may move to adjacent numbers on the row below.

<font color='dd0000'>注意点：①.二维到一维的空间优化；②.自底向上比自顶向下效率更高</font>

In [5]:
// 1. 自顶向下
// 贪心的在每一步选择最小和无法得到全局最优解
// 因此，使用动态规划，每一层的每个位置都记录当前的路径和，在最后一层中最小的值即为结果,所以是一个二维dp
// 状态设计：dp[i][j] 表示 第i层的第j个位置的路径和
// 状态转移：dp[i][j] = min(dp[i-1][j-1], dp[i-1][j]) + nums[i][j]，需要特别考虑第一个和最后一个元素

public int minimumTotal11(List<List<Integer>> triangle) {
    int n = triange.length;
    int[][] dp = new int[n][n];
    dp[0][0] = triangle.get(0).get(0);
    for(int i = 1; i < n; i++){
        for(int j = 0; j < triangle.get(i).size(); j++){
            if(j == 0){  // 第一个元素上一层没有j-1
                dp[i][j] = dp[i-1][j];
            } else if(j == triangle.get(i).size() - 1){  // 最后一个元素上一层没有j
                dp[i][j] = dp[i-1][j-1];
            } else {
                dp[i][j] = Math.min(dp[i-1][j], dp[i-1][j-1]);
            }
            dp[i][j] += triangle.get(i).get(j);
        }
    }
    // 在最后一层中找最小值
    int res = dp[n-1][0];
    for(int j = 1; j < n; j++){
        if(dp[n-1][j] < res){
            res = dp[n-1][j];
        }
    }
    return res;
}

// 可以看出每次状态转移只与上一层结果有关，所以可以用一维滚动数组进行空间优化
public int minimumTotal12(List<List<Integer>> triangle) {
    int n = triangle.size();
    int[] dp = new int[n];
    dp[0] = triangle.get(0).get(0);
    for(int i = 1; i < n; i++){
        for(int j = triangle.get(i).size() - 1; j >= 0; j--){ 
            if(j == 0){
                dp[j] = dp[j];
            } else if(j == triangle.get(i).size() - 1){
                dp[j] = dp[j-1];
            } else {
                dp[j] = Math.min(dp[j], dp[j-1]);
            }
            dp[j] += triangle.get(i).get(j);
        }
    }
    int res = dp[0];
    for(int j = 1; j < n; j++){
        if(dp[j] < res){
            res = dp[j];
        }
    }
    return res;
}

In [6]:
// 2.自底向上
// 从最后一层为起点，向上递推，每一步同样取最小路径和，这样最后仅有的dp[0]就是最终结果，更简洁
// 状态转移：dp[i][j] = min(dp[i+1][j], dp[i+1][j+1])+nums[i][j]，无需考虑边界情况
// 同样可以用滚动数组进行空间优化
public int minimumTotal2(List<List<Integer>> triangle) {
    int n = triangle.size();
    int[] dp = new int[n];
    for(int j = 0; j < n; j++){
        dp[j] = triangle.get(n-1).get(j);
    }
    for(int i = n - 2; i >= 0; i--){
        for(int j = 0; j < triangle.get(i).size(); j++){
            dp[j] = Math.min(dp[j], dp[j+1]) + triangle.get(i).get(j);
        }
    }
    return dp[0];
}

## [53. Maximum Subarray (easy)](https://leetcode-cn.com/problems/maximum-subarray/)

Given an integer array ```nums```, find the contiguous subarray (containing at least one number) which has the largest sum and return its sum.

In [10]:
// dp 空间优化
public int maxSubArray(int[] nums) {
    int n = nums.length;
    if(n < 1){
        return Integer.MIN_VALUE;
    }
    int sum = nums[0]; // 一维dp数组空间优化到常数
    int res = sum;
    for(int i = 1; i < n; i++){
        sum = Math.max(sum + nums[i], nums[i]);
        res = Math.max(res, sum);
    }
    return res;
}

// 分治
public int maxSubArray(int[] nums) {
    return split(nums, 0, nums.length - 1);
}

private int split(int[] nums, int left, int right){
    if(left == right){
        return nums[left];
    }
    int mid = left + (right - left) / 2;
    int l = split(nums, left, mid);
    int r = split(nums, mid + 1, right);
    int cross = merge(nums, left, right, mid);
    return Math.max(l, Math.max(r, cross));
}

private int merge(int[] nums, int left, int right, int mid){ //求跨过中间的最大子序列和
    int lm = Integer.MIN_VALUE, rm = Integer.MIN_VALUE;
    int cur = 0;
    for(int i = mid; i >= left; i--){  // 从中间向左扫描
        cur += nums[i];
        lm = Math.max(lm, cur);
    }
    cur = 0;
    for(int i = mid + 1; i <= right; i++){ // 向右扫描
        cur += nums[i];
        rm = Math.max(rm, cur);
    }
    return lm + rm;
}

## [152. Maximum Product Subarray (medium)](https://leetcode-cn.com/problems/maximum-product-subarray/)

Given an integer array ```nums```, find the contiguous subarray within an array (containing at least one number) which has the largest product.

<font color='dd0000'>

因为乘积存在负负得正的情况，所以不能像上题一样，在每一步记录最大值，可能后续的大值是用前面的小值乘负数得到的

所以在每一步应同时记录最大值和最小值，$dpMax[]$ 和 $dpMin[]$ 

同时，根据观察可知，第 $i$ 步的最大值和最小值只有三种情况，即为状态转移，这样不必再讨论当前值的正负：

1. ```dpMax[i] = max(dpMax[i-1] * nums[i], dpMin[i-1] * nums[i], nums[i])```

2. ```dpMin[i] = min(dpMin[i-1] * nums[i], dpMax[i-1] * nums[i], nums[i])```

同样，可以用滚动变量进行空间优化

</font>

In [8]:
public int maxProduct(int[] nums) {
    int n = nums.length;
    int[] dpMax = new int[n];
    int[] dpMin = new int[n];
    dpMax[0] = nums[0];
    dpMin[0] = nums[0];
    int res = dpMax[0];
    for(int i = 1; i < n; i++){
        dpMax[i] = Math.max(dpMax[i-1]*nums[i], Math.max(nums[i], dpMin[i-1]*nums[i]));
        dpMin[i] = Math.min(dpMin[i-1]*nums[i], Math.min(nums[i], dpMax[i-1]*nums[i]));
        res = Math.max(res, dpMax[i]);
    }
    return res;
}

// 优化
public int maxProduct(int[] nums) {
    int res = nums[0]; 
    int max = 1, min = 1;
    for(int i = 0; i < nums.length; i++){
        if(nums[i] < 0){   // 因为当nums[i]为负时，乘出来的结果会大的变成小的，小的变成大的
            int tmp = max; // 所以交换max和min值，这样方便后面选择最大时用一个表达式即可
            max = min;
            min = tmp;
        }
        max = Math.max(max * nums[i], nums[i]);
        min = Math.min(min * nums[i], nums[i]);
            
        res = Math.max(res, max);
    }
    return res;
}

## [983. 最低票价 (medium)](https://leetcode-cn.com/problems/minimum-cost-for-tickets/)

[<font color='dd0000'>思路</font>](https://leetcode-cn.com/problems/minimum-cost-for-tickets/solution/java-dong-tai-gui-hua-si-lu-bu-zou-cong-hou-xiang-/)

In [1]:
/* 
   因为当天如何买票，要取决于后面的天数怎么安排，所以从后往前推
   状态：dp[i] 表示 在从第i天到结束，所需的总最低花费
   转移：如果第i天要出行：dp[i] = min{dp[i+1] + costs[0], dp[i+7] + costs[1], dp[i+30] + costs[2]}
                                     (买一天票,包当天), (买7天票,包后面7天), (买30天票,包后面30天)
  
         如果第i天不出行：dp[i] = dp[i+1]
                          (和后一天的花费相同)     
*/

public int mincostTickets(int[] days, int[] costs) {
    int n = days.length;
    int start = days[0], end = days[n-1];  // 旅行的起始和结束日期，在这个范围内递推
    int[] dp = new int[end + 31]; // 防止处理后面的天数时越界
    int index = n - 1; // index用来表示出行日的下标
    // 从后往前递推
    for(int i = end; i >= start; i--){
        if(i != days[index]){
            dp[i] = dp[i + 1];   // 不出行
        } else {
            int c1 = dp[i + 1] + costs[0];
            int c2 = dp[i + 7] + costs[1];
            int c3 = dp[i + 30] + costs[2];
            dp[i] = Math.min(c1, Math.min(c2, c3));
            index--;
        }
    }
    return dp[start];  // 返回从开始日到结束的总最低花费
}

## [221. Maximal Square (medium)](https://leetcode-cn.com/problems/maximal-square/)

Given a 2D binary matrix filled with 0's and 1's, find the largest square containing only 1's and return its area.

[<font color='dd0000'>思路</font>](https://leetcode-cn.com/problems/maximal-square/solution/li-jie-san-zhe-qu-zui-xiao-1-by-lzhlyle/)

In [2]:
/*
  状态：dp[i][j] 表示 以matrix[i][j]为右下角的正方形的最大边长
  转移：dp[i][j] = max{dp[i-1][j-1], dp[i-1][j], dp[i][j-1]} + 1
*/

public int maximalSquare(char[][] matrix) {
    int m = matrix.length;
    if(m == 0){
        return 0;
    }
    int n = matrix.length;
    if(n == 0){
        return 0;
    }
    int[][] dp = new int[m][n];
    int max = 0;
    // 初始化第一行和第一列
    for(int i = 0; i < m; i++){
        if(matrix[i][0] == '1'){
            dp[i][0] = 1;
            max = 1;
        }
    }
    for(int j = 0; j < n; j++){
        if(matrix[0][j] == '1'){
            dp[0][j] = 1;
            max = 1;
        }
    }
    // 递推
    for(int i = 1; i < m; i++){
        for(int j = 1; j < n; j++){
            if(matrix[i][j] == '1'){
                dp[i][j] = Math.min(dp[i-1][j-1], Math.min(dp[i-1][j], dp[i][j-1])) + 1;
                max = Math.max(max, dp[i][j]);
            }
        }
    }
    return max * max;  // 返回面积
}

## [887. 鸡蛋掉落 (hard)](https://leetcode-cn.com/problems/super-egg-drop/)

你将获得 ```K``` 个鸡蛋，并可以使用一栋从 ```1``` 到 ```N```  共有 ```N``` 层楼的建筑。

每个蛋的功能都是一样的，如果一个蛋碎了，你就不能再把它掉下去。

你知道存在楼层 ```F``` ，满足 ```0 <= F <= N``` 任何从高于 ```F``` 的楼层落下的鸡蛋都会碎，从 ```F``` 楼层或比它低的楼层落下的鸡蛋都不会破。

每次移动，你可以取一个鸡蛋（如果你有完整的鸡蛋）并把它从任一楼层 ```X``` 扔下（满足 ```1 <= X <= N```）。

你的目标是**确切地**知道 ```F``` 的值是多少。

无论 ```F``` 的初始值如何，你确定 ```F``` 的值的最小移动次数是多少？

[<font color='dd0000'>**题解**](https://leetcode-cn.com/problems/super-egg-drop/solution/ji-ben-dong-tai-gui-hua-jie-fa-by-labuladong/)

## [837. 新21点 (medium)](https://leetcode-cn.com/problems/new-21-game/)

爱丽丝参与一个大致基于纸牌游戏 “21点” 规则的游戏，描述如下：

爱丽丝以 ```0``` 分开始，并在她的得分少于 ```K``` 分时抽取数字。 抽取时，她从 ```[1, W]``` 的范围中随机获得一个整数作为分数进行累计，其中 ```W``` 是整数。 每次抽取都是独立的，其结果具有相同的概率。

当爱丽丝获得不少于 ```K``` 分时，她就停止抽取数字。 爱丽丝的分数不超过 ```N``` 的概率是多少？

<font color='dd0000'>

当手中的点数**大于等于** $K$ 时，停止抽取。求的是当停止抽取后，手中点数**小于等于** $N$ 的概率

用 $f(x)$ 表示，当前手中点数为 $x$ 时，到抽取结束，最后数字之和小于等于 $N$ 的概率

所以手中的点数为 $x$ 且抽取已经停止时(即此时 $x >= K$)，可知当 $K <= x <= N$ 时，有 $f(x) = 1$，当 $x > N$ 时，$f(x) = 0$

考虑 $x < K$ 时，还需要若干次抽取才能停止，每次抽取的结果都会使点数变为 $x+1, x+2, ..., x+W$ ，即成为更接近停止的状态

**所以可以进行倒推的动态规划**

**状态设计：** $dp[i]$ 表示当前手中点数为 $i$ 时，到抽取结束，最后数字之和小于等于 $N$ 的概率

**状态转移：** $dp[i] = \frac{1}{W}(dp[i+1] + dp[i+2] + ... +dp[i+W])$ 因为转移到每一个新状态的概率为 $\frac{1}{W}$ 新状态又表示结果小于等于的概率

**初始化：** 当 $i >= K$ 时，根据最大值 $K-1+W$ 和 $N$ 的大小关系，设为1或0

**优化：** 如果每次转移时都写出上述转移公式，时间复杂度为 $O(KW)$，可能超时。观察转移方程，可对相邻两项作差化简，去掉内循环

$dp[i] = \frac{1}{W}(dp[i+1] + dp[i+2] + ... + dp[i+W-1] + dp[i+W])$

$dp[i - 1] = \frac{1}{W}(dp[i] + dp[i+1] + dp[i+2] ... + dp[i+W-1])$

所以，$dp[i-1] = \frac{1}{W}(dp[i] - dp[i+W]) + dp[i]$

即： $dp[i] = \frac{1}{W}(dp[i+1] - dp[i+W+1]) + dp[i+1]$。当 $i=K-1$ 时，$dp[i+W+1])$会越界，所以单独初始化

</font>

In [1]:
public double new21Game(int N, int K, int W) {
    double[] dp = new double[K + W]; // 最大抽到的点数为K - 1 + W, 所以数组长度为K + W
    // 初始化
    if(K - 1 + W <= N){
        for(int i = K; i < K + W; i++){
            dp[i] = 1.0;
        }
    } else {
        for(int i = K; i <= N; i++){
            dp[i] = 1.0;
        }
    }
    // 从 K-1 开始逆向递推
    for(int i = K - 1; i >= 0; i--){
        for(int j = 1; j <= W; j++){
            dp[i] += (1.0 / W) * dp[i + j];
        }
    }
    return dp[0];
}

// 时间优化
public double new21Game(int N, int K, int W) {
    double[] dp = new double[K + W]; // 最大抽到的点数为K - 1 + W, 所以数组长度为K + W
    // 初始化
    if(K - 1 + W <= N){
        for(int i = K; i < K + W; i++){
            dp[i] = 1.0;
        }
    } else {
        for(int i = K; i <= N; i++){
            dp[i] = 1.0;
        }
    }
    for(int j = 1; j <= W; j++){
        dp[K - 1] += (1.0 / W) * dp[K - 1 + j];  // K-1单独考虑
    }
    // 从 K-2 开始逆向递推
    for(int i = K - 2; i >= 0; i--){
        dp[i] = (1.0 / W) * (dp[i + 1] - dp[i + W + 1]) + dp[i + 1];
    }
    return dp[0];
}

## [剑指 Offer 46. 把数字翻译成字符串(medium)](https://leetcode-cn.com/problems/ba-shu-zi-fan-yi-cheng-zi-fu-chuan-lcof/)

给定一个数字，我们按照如下规则把它翻译为字符串：0 翻译成 “a” ，1 翻译成 “b”，……，11 翻译成 “l”，……，25 翻译成 “z”。一个数字可能有多个翻译。请编程实现一个函数，用来计算一个数字有多少种不同的翻译方法。

<font color='dd0000'>

本质上和**上楼梯**是一样的。对第 $i$ 位数，如果第 $i-1$ 位和它能组成26以内的数，那么它就有两种翻译法，否则就只有一种

所以第 $i$ 位数可能从第 $i-2$ 和第 $i-1$ 两种状态转移过来

**状态设计：**$dp[i]$ 表示到第 $i$ 位数为止所有的翻译方法

**状态转移：**根据条件，$dp[i] = dp[i-1]$ 或 $dp[i] = dp[i-1] + dp[i-2]$

</font>

In [ ]:
public int translateNum(int num) {
    String s = String.valueOf(num);
    if(s.length() == 1){
        return 1;
    }
    char[] chars = s.toCharArray();
    int n = chars.length;
    int[] dp = new int[n];
    // 初始化
    dp[0] = 1;
    if((chars[0] == '2' && chars[1] <= '5') || chars[0] == '1'){
        dp[1] = 2;
    } else {
        dp[1] = 1;
    }
    // 递推
    for(int i = 2; i < n; i++){
        if((chars[i - 1] == '2' && chars[i] <= '5') || chars[i - 1] == '1'){
            dp[i] = dp[i - 2] + dp[i - 1];
        } else{
            dp[i] = dp[i - 1];
        }
    }
    return dp[n - 1];
}

## [1014. 最佳观光组合 (medium)](https://leetcode-cn.com/problems/best-sightseeing-pair/)

给定正整数数组 ```A```，```A[i]``` 表示第 ```i``` 个观光景点的评分，并且两个景点 ```i``` 和 ```j``` 之间的距离为 ```j - i```。

一对景点（```i < j```）组成的观光组合的得分为（```A[i] + A[j] + i - j```）：景点的评分之和**减去**它们两者之间的距离。

返回一对观光景点能取得的最高分。

<font color='dd0000'>

所求结果为式子```A[i] + A[j] + i - j```对所有```i,j (i < j)```的最大值

将式子变形为```A[i] + i + A[j] - j, (i < j)```，即对一个固定的```j```找它前面的```i```中```A[i] + i```的最大值

</font>

In [ ]:
public int maxScoreSightseeingPair(int[] A) {
    int n = A.length;
    // 先求出截止到每个i, 最大的A[i]+i
    int[] iMax = new int[n];
    iMax[0] = A[0];
    for(int i = 1; i < n; i++){
        iMax = Math.max(iMax[i - 1], A[i] + i);
    }
    // 遍历j，求最大值
    int res = 0;
    for(int j = 1; j < n; j++){
        res = Math.max(res, A[j] - j + iMax[j]);
    }
    return res;
}

// 空间优化：不需要用数组保存每个位置A[i] + i的最大值，用一个变量记录j之前的最大值即可，一一边遍历一边计算
 public int maxScoreSightseeingPair(int[] A) {
    int n = A.length;
    int res = 0;
    int maxi = A[0];
    for(int j = 1; j < n; j++){
        res = Math.max(res, A[j] - j + maxi);
        maxi = Math.max(maxi, A[j] + j); // 更新最大值
    }
    return res;
}

## ⭐[10. Regular Expression Matching (hard)](https://leetcode-cn.com/problems/regular-expression-matching/)

Given an input string (```s```) and a pattern (```p```), implement regular expression matching with support for ```'.'``` and ```'*'```.

<pre>'.' Matches any single character.
'*' Matches zero or more of the preceding element.</pre>

The matching should cover the entire input string (not partial).

**Note:**

- ```s``` could be empty and contains only lowercase letters ```a-z```.
- ```p``` could be empty and contains only lowercase letters ```a-z```, and characters like ```.``` or ```*```.

<font color='dd0000'>

和 <a href="#⭐72.-Edit-Distance-(hard)">72.编辑距离</a> 类似，涉及到两个字符串的匹配问题，一般使用二维的dp.

</font>

![](010.png)

In [ ]:
public boolean isMatch(String s, String p) {
    char[] S = s.toCharArray();
    char[] P = p.toCharArray();
    int m = S.length, n = P.length;
    boolean[][] dp = new boolean[m + 1][n + 1];
    // 初始化
    dp[0][0] = true;
    for(int j = 1; j <= n; j++){ // p匹配空s的情况
        if(P[j - 1] == '*'){
            dp[0][j] = dp[0][j - 2];
        }
    }
    // 递推
    for(int i = 1; i <= m; i++){
        for(int j = 1; j <= n; j++){
            if(S[i - 1] == P[j - 1] || P[j - 1] == '.'){ //对应字符匹配,取决于前i-1位是否匹配
                dp[i][j] = dp[i - 1][j - 1];
            } else if(P[j - 1] == '*'){ //当j是*时，也可能匹配
                if(P[j - 2] != S[i - 1] && P[j - 2] != '.'){ //p的前一位不是和s不同,要匹配只能*取0
                    dp[i][j] = dp[i][j - 2];
                } else{ // p的前一位和s当前位相同或为.,*的取值可能有多个,根据推导:
                    dp[i][j] = dp[i][j - 2]  // *取0
                            || (dp[i-1][j-2] || dp[i-1][j] && (S[i-2] == P[j-2] || P[j-2] == '.'));//*>=1
                }
            }
        }
    }
    return dp[m][n];
}

## [44. Wildcard Matching (hard)](https://leetcode-cn.com/problems/wildcard-matching/)

Given an input string (```s```) and a pattern (```p```), implement wildcard pattern matching with support for '```?```' and '```*```'.

```
'?' Matches any single character.
'*' Matches any sequence of characters (including the empty sequence).
```
**The matching should cover the entire input string (not partial).**

**Note:**

- ```s``` could be empty and contains only lowercase letters ```a-z```.
- ```p``` could be empty and contains only lowercase letters ```a-z```, and characters like ```?``` or ```*```.

<font color='dd0000'>和上题 <a href="#⭐10.-Regular-Expression-Matching-(hard)">10.正则表达式匹配</a> 思路相同</font>

<font color='dd0000'>(贪心解法 参照题解)</font>

In [ ]:
public boolean isMatch(String s, String p) {
    // 在两个字符串前填充空字符，避免下标混乱
    s = " " + s;
    p = " " + p;
    char[] S = s.toCharArray();
    char[] P = p.toCharArray();
    int n = S.length, m = P.length;
    boolean[][] dp = new boolean[m][n];
    // 初始化
    dp[0][0] = true; // 空字符和空字符一定匹配,空字符一定不匹配其他字符,所以第一行只有第一个位置为true
    for(int i = 1; i < m; i++){ // 对第一列,S为只有当P[i]为'*'时有可能匹配
        if(P[i] == '*'){
            dp[i][0] = dp[i - 1][0]; // 为'*'时,'*'表示空,看前面是否为空即可(即是否匹配)
        }
    }
    // 递推
    for(int i = 1; i < m; i++){
        for(int j = 1; j < n; j++){
            if(P[i] == S[j] || P[i] == '?'){
                dp[i][j] = dp[i - 1][j - 1];
            } else if (P[i] == '*'){
                dp[i][j] = dp[i - 1][j] || dp[i][j - 1]; // 列式推导或逻辑推理两种思路
            }
        }
    }
    return dp[m - 1][n - 1];
}

## [139. Word Break (medium)](https://leetcode-cn.com/problems/word-break/)

Given a **non-empty** string s and a dictionary wordDict containing a list of **non-empty** words, determine if s can be segmented into a space-separated sequence of one or more dictionary words.

**Note:**

- The same word in the dictionary may be reused multiple times in the segmentation.
- You may assume the dictionary does not contain duplicate words.

<font color='dd0000'>**动态规划**</font>

In [ ]:
// 用dp[i]表示前i个字符能否被拆分
// 每次递推时，依次枚举检查

public boolean wordBreak(String s, List<String> wordDict) {
    Set<String> set = new HashSet<>(wordDict);
    int n = s.length();
    // 求字典中字符串的最大长度,可进行剪枝
    int maxLen = 0;
    for(String str : set){
        maxLen = Math.max(maxLen, str.length());
    }
    boolean[] dp = new boolean[n];
    for(int i = 0; i < n; i++){
        for(int j = i; j >= Math.max(0, i - maxLen + 1); j--){ // 每次不一定非要检查到0,当子串超过maxLen时就可以跳出了
            String sub = s.substring(j, i + 1);  // 从j到i的子串
            if(j == 0){
                if(set.contains(sub)){
                    dp[i] = true;
                    break;    // 如果检查到能拆分,就直接跳出循环
                }
            } else {
                if(set.contains(sub) && dp[j - 1]){
                    dp[i] = true;
                    break;
                }
            }
        }
    }
    return dp[n - 1];
}

## [718. Maximum Length of Repeated Subarray (medium)](https://leetcode-cn.com/problems/maximum-length-of-repeated-subarray/)

Given two integer arrays ```A``` and ```B```, return the maximum length of an subarray that appears in both arrays.

<font color='dd0000'>

**方法一：** 二维动态规划(两个字符串、数组的匹配问题)

**方法二：** 滑动窗口⭐ [<font color='dd00dd'>题解](https://leetcode-cn.com/problems/maximum-length-of-repeated-subarray/solution/wu-li-jie-fa-by-stg-2/)

</font>

In [ ]:
// dp[i][j] 表示包括A[i]和B[j]的两个子数组最大公共长度
// 如果 A[i] == B[j], dp[i][j] = dp[i - 1][j - 1] + 1, 即前一位的匹配长度加上当前这个匹配的数字
// 如果不相等, 则最大长度为0

public int findLength(int[] A, int[] B) {
    int m = A.length, n = B.length;
    int[][] dp = new int[m + 1][n + 1];
    int res = 0;
    for(int i = 1; i <= m; i++){
        for(int j = 1; j <= n; j++){
            if(A[i - 1] == B[j - 1]){
                dp[i][j] = dp[i - 1][j - 1] + 1;
                res = Math.max(res, dp[i][j]);
            }
        }
    }
    return res;
}

// 空间优化
public int findLength(int[] A, int[] B) {
    int m = A.length;
    int n = B.length;
    int[] dp = new int[n + 1];
    int res = 0;
    for(int i = 1; i <= m; i++){
        for(int j = n; j >= 1; j--){ // 因为后面的j要依赖前面的j 所以逆序
            if(A[i - 1] == B[j - 1]){
                dp[j] = dp[j - 1] + 1;
                res = Math.max(res, dp[j]);
            }
        }
    }
    return res;
}

In [5]:
// 滑动窗口
public int findLength(int[] A, int[] B) {
    if(A.length < B.length) return slideWindow(A, B);
    return slideWindow(B, A);
}

// 默认A的长度更小
private int slideWindow(int[] A, int[] B){
    int res = 0;  // 保存结果，即最大匹配长度
    int n1 = A.length, n2 = B.length;
    // 第一次滑动：固定A，把B向右滑，每次从A的起点开始和B进行匹配，直到A B起点重合
    for(int len = 1; len <= n1; len++){
        res = Math.max(res, getMaxLen(A, 0, B, n2 - len, len)); 
    }
    // 上面的循环结束后, A B起点重合, 当n1小于n2时 B头部还剩一段
    for(int index = n2 - n1 - 1; index >= 0; index--){
        res = Math.max(res, getMaxLen(A, 0, B, index, n1)); // 匹配长度最大就是A的长度
    }
    // 第二次滑动：固定B，把A向左滑，每次从B的起点开始和A进行匹配，直到A的尾部和B的开头重合
    for(int len = n1 - 1; len > 0; len--){
        res = Math.max(res, getMaxLen(A, n1 - len, B, 0, len));
    }
    return res;
}

// 求当前匹配的最大长度
// 参数: 数组A, A的起点, 数组B，B的起点，要匹配的长度
private int getMaxLen(int[] A, int startA, int[] B, int startB, int len){
    int max = 0;
    int count = 0;
    for(int i = 0; i < len; i++){
        if(A[startA + i] == B[startB + i]){
            count++;
            max = Math.max(max, count);
        } else if(count > 0){
            count = 0;
        }
    }
    return max;
}

## [343. Integer Break (medium)](https://leetcode-cn.com/problems/integer-break/)

Given a positive integer n, break it into the sum of **at least** two positive integers and maximize the product of those integers. Return the maximum product you can get.

In [1]:
// 动态规划
public int integerBreak(int n) {
    int[] dp = new int[n + 1];
    dp[1] = 1;
    dp[2] = 1;
    for(int i = 3; i <= n; i++){
        int temp = 0;
        for(int j = 1; j < n / 2; j++){
            temp = Math.max(i * dp[i - j], i * (i - j));
        }
        dp[i] = temp;
    }
    return dp[n];
}

[<font color='dd0000'>数学推导</font>](https://leetcode-cn.com/problems/integer-break/solution/343-zheng-shu-chai-fen-tan-xin-by-jyd/): 将数字尽可能分成更多的3，再对余数进行处理

In [2]:
// 数学推导
public int integerBreak(int n) {
    if(n <= 3){
        return n - 1;
    }
    int numOf3 = n / 3;
    int remain = n % 3;
    if(remain == 0){
         return (int) Math.pow(3, numOf3);
    }
    else if(remain == 1){ // 余数为1，分出一个3来组成4
        return (int) Math.pow(3, numOf3 - 1) * 4;
    }
    else{ // 余数为2，直接相乘
        return (int) Math.pow(3, numOf3) * 2;
    }
}

<a name="bottom"></a>
<p style="text-align: right"><a href="#top" style="text-decoration: none;color: pink"><strong>Top</strong></a></p>